In [245]:
import keras

In [246]:
import os
import numpy
# from keras.datasets import patient
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.models import model_from_json

import pandas
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from io import StringIO

# fix random seed for reproducibility
numpy.random.seed(7)

In [247]:
pandas.set_option('display.max_colwidth', 1000)
print(os.getcwd())
# dataset = pandas.read_csv(StringIO(s), quotechar='"', skipinitialspace=True, error_bad_lines = False)
dataset = pandas.read_csv('record.csv')
dataset = pandas.DataFrame(dataset)
dataset = dataset.dropna(axis=0, how='any')
X = dataset['symptoms']
Y = dataset['disease']

# encode class values as integers
encoder_Y = LabelEncoder()
docs2 = ["arthritis","asthma","ASD","cancer","chlamydia","ebola","diabetes","malaria","HIV","cirrhosis","migraine", 
        "heart disease","rhinovirus","influenza","stroke","alzheimer disease","tuberculosis","thyroid"]
encoder_Y.fit(docs2)
encoded_Y = encoder_Y.transform(Y)
print(encoded_Y[2])
# convert integers to dummy variables (i.e. one hot encoded)
Y = np_utils.to_categorical(encoded_Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

C:\Users\SiSharma\AnacondaProjects
0        ['muscle aches', 'weakness', 'sore throat', 'high fever', 'coughing', 'headache']
1                                             ['pain', 'swelling', 'stiffness', 'redness']
2                        ['blood discharge', 'coughing', 'fever', 'fatigue', 'chest pain']
3                                                ['jaundice', 'weight loss', 'nose bleed']
4                                             ['pain during urination', 'blood discharge']
5                                          ['swelling', 'blood discharge', 'chronic pain']
6                                                         ['memory loss', 'blurry vision']
7               ['muscle aches', 'chills', 'nausea', 'vomiting', 'high fever', 'headache']
8                                             ['pain', 'swelling', 'stiffness', 'redness']
9                              ['palpitation', 'weakness', 'faster heartbeat', 'sweating']
10              ['muscle aches', 'chills', 'nausea', 'v

In [236]:

# define 5 documents
docs = ["headache","nausea","vomiting","high fever","diarrhea","fatigue","muscle aches","coughing","pain","stiffness",
        "swelling","redness","wheezing","learning disability","epilepsy","chronic pain","blood discharge","sneezing",
        "pain during urination","genital pain","palpitation","weakness","faster heartbeat", "nasal dryness","sore throat",
        "abdominal pain","weight loss","chills","chest pain","back pain","tired","blindness","blurred vision","dry mouth",
        "paralysis","dizziness","memory loss","nose bleed","jaundice","constipation"]

vocab_size = 70
max_review_length = 200
MAX_NUM_WORDS = 1000
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(docs)
# summarize
print(vectorizer.vocabulary_)
# encode document
X_train_vector = vectorizer.transform(X_train)
X_test_vector = vectorizer.transform(X_test)
X_train =  X_train_vector.toarray()
X_test =  X_test_vector.toarray()
#padding the data
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_review_length, dtype='object', 
                                           padding='pre', truncating='pre', value=0.)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_review_length, dtype='object', 
                                           padding='pre', truncating='pre', value=0.)


{'headache': 24, 'nausea': 34, 'vomiting': 48, 'high': 26, 'fever': 22, 'diarrhea': 12, 'fatigue': 21, 'muscle': 32, 'aches': 1, 'coughing': 11, 'pain': 36, 'stiffness': 42, 'swelling': 43, 'redness': 39, 'wheezing': 51, 'learning': 28, 'disability': 13, 'epilepsy': 19, 'chronic': 9, 'blood': 5, 'discharge': 14, 'sneezing': 40, 'during': 18, 'urination': 46, 'genital': 23, 'palpitation': 37, 'weakness': 49, 'faster': 20, 'heartbeat': 25, 'nasal': 33, 'dryness': 17, 'sore': 41, 'throat': 44, 'abdominal': 0, 'weight': 50, 'loss': 29, 'chills': 8, 'chest': 7, 'back': 2, 'tired': 45, 'blindness': 4, 'blurred': 6, 'vision': 47, 'dry': 16, 'mouth': 31, 'paralysis': 38, 'dizziness': 15, 'memory': 30, 'nose': 35, 'bleed': 3, 'jaundice': 27, 'constipation': 10}


AttributeError: 'Series' object has no attribute 'getType'

In [224]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(200, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(18, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 200, 32)           1280      
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_9 (Dense)              (None, 18)                1818      
Total params: 56,298
Trainable params: 56,298
Non-trainable params: 0
_________________________________________________________________
None
Train on 79999 samples, validate on 20000 samples
Epoch 1/3
79999/79999 [==============================] - 205s 3ms/step - loss: 0.7671 - acc: 0.7807 - val_loss: 0.4270 - val_acc: 0.8571
Epoch 2/3
79999/79999 [==============================] - 202s 3ms/step - loss: 0.4735 - acc: 0.8462 - val_loss: 0.4210 - val_acc: 0.8571
Epoch 3/3
79999/79999 [==============================] - 202s 3ms/step - 

In [225]:
# Final evaluation of the model
scores = model.evaluate(X_train, Y_train, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.72%


In [226]:
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.71%


In [228]:
# serialize model to JSON
model_json = model.to_json()
with open("D:\\Hackathon\\trained_model\\model_categorical.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("D:\\Hackathon\\trained_model\\model_categorical.h5")
print("Saved model to disk")
 
# later...
 
# # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
 
# # evaluate loaded model on test data
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Saved model to disk


ValueError: Error when checking : expected embedding_2_input to have shape (500,) but got array with shape (1,)